# Text classification 

In [1]:
import re
import xml.sax.saxutils as saxutils

#from BeautifulSoup import BeautifulSoup

from gensim.models.word2vec import Word2Vec

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.layers.convolutional import Conv1D, MaxPooling1D, AveragePooling1D
from keras.layers.core import Flatten #, Dense, Dropout
from keras.optimizers import Adam

#from multiprocessing import cpu_count

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer

from pandas import DataFrame

from sklearn.cross_validation import train_test_split

C:\Anaconda2\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 760 (CNMeM is disabled, cuDNN not available)
C:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", Depre

# Setup constants 

In [2]:
# Set Numpy random seed
import random
random.seed(1000)

# data and model folder
data_folder = '.\\data\\'

# Word2Vec number of features
num_features = 500
# Limit each newsline to a fixed number of words
document_max_num_words = 100

# Prepare content

In [3]:
import json

with open(data_folder + 'content.json', 'r') as f:
     content = json.load(f)

with open(data_folder + 'label.json', 'r') as f:
     label = json.load(f)

print label['1001'], 'topic, place, people, organization, exchanges'
print content['1001']

[0, 1, 0, 0, 0] topic, place, people, organization, exchanges
sandoz ag said it planned a joint venture
to produce herbicides in the soviet union.
    the company said it had signed a letter of intent with the
soviet ministry of fertiliser production to form the first
foreign joint venture the ministry had undertaken since the
soviet union allowed western firms to enter into joint ventures
two months ago.
    the ministry and sandoz will each have a 50 pct stake, but
a company spokeswoman was unable to give details of the size of
investment or planned output.
 


# Tokenize contents

In [4]:
# Load stop-words
stop_words = set(stopwords.words('english'))

# Initialize tokenizer
tokenizer = RegexpTokenizer('[\'a-zA-Z]+')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Tokenized content collection
content_documents = []

In [6]:
import nltk

nltk.download('punkt')

def tokenize(document):
    words = []
    for sentence in sent_tokenize(document):
        tokens = [lemmatizer.lemmatize(t.lower()) for t in tokenizer.tokenize(sentence) if t.lower() not in stop_words]
        words += tokens

    return words

content_documents = []
# Tokenize
key1001 = 0
for key in content.keys():
    if key == '1001':
        key1001 = len(content_documents)
        print len(content_documents)
        print tokenize(content[key])
    content_documents.append(tokenize(content[key]))

number_of_documents = len(content)

[nltk_data] Downloading package punkt to C:\Anaconda2\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
17743
[u'sandoz', u'ag', u'said', u'planned', u'joint', u'venture', u'produce', u'herbicide', u'soviet', u'union', u'company', u'said', u'signed', u'letter', u'intent', u'soviet', u'ministry', u'fertiliser', u'production', u'form', u'first', u'foreign', u'joint', u'venture', u'ministry', u'undertaken', u'since', u'soviet', u'union', u'allowed', u'western', u'firm', u'enter', u'joint', u'venture', u'two', u'month', u'ago', u'ministry', u'sandoz', u'pct', u'stake', u'company', u'spokeswoman', u'unable', u'give', u'detail', u'size', u'investment', u'planned', u'output']


# Word2Vec Model

In [7]:
# Create new Gensim Word2Vec model
w2v_model = Word2Vec(content_documents, size=num_features, min_count=1, window=10)
w2v_model.init_sims(replace=True)

# Vectorize content

In [8]:
import numpy

num_categories = 5
X = numpy.zeros(shape=(number_of_documents, document_max_num_words, num_features)).astype(numpy.float32)
Y = numpy.zeros(shape=(number_of_documents, num_categories)).astype(numpy.float32)

empty_word = numpy.zeros(num_features).astype(numpy.float32)

for idx, document in enumerate(content_documents):
    for jdx, word in enumerate(document):
        if jdx == document_max_num_words:
            break
            
        else:
            if word in w2v_model:
                X[idx, jdx, :] = w2v_model[word]
            else:
                X[idx, jdx, :] = empty_word

for idx, key in enumerate(label.keys()):
    Y[idx, :] = label[key]
    
print X[key1001]

[[-0.02157213 -0.04568234  0.00146626 ..., -0.00972886 -0.01741979
  -0.00713177]
 [-0.03147293 -0.0149641   0.03508525 ..., -0.03686867 -0.00689081
  -0.02585261]
 [-0.02510744 -0.00800088 -0.00946434 ...,  0.08622478  0.01505131
  -0.05790702]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


# Split training and test sets

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

# Create Keras model

In [10]:
model = Sequential()

filters = document_max_num_words
kernel_size = 5
model.add(Conv1D(filters, kernel_size, activation='elu', border_mode='same', input_shape=(document_max_num_words, num_features)))
model.add(Conv1D(filters, kernel_size, activation='elu', border_mode='same'))
model.add(AveragePooling1D(3))
model.add(Dropout(0.25))

filters = document_max_num_words*2
kernel_size = 2
model.add(Conv1D(filters, kernel_size, activation='elu', border_mode='same'))
model.add(Conv1D(filters, kernel_size, activation='elu', border_mode='same'))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256, activation='tanh'))
model.add(Dense(256, activation='tanh'))
model.add(Dropout(0.5))

model.add(Dense(num_categories, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

C:\Anaconda2\lib\site-packages\theano\tensor\basic.py:2146: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"


# Train and evaluate model

In [11]:
# Train model
model.fit(X_train, Y_train, batch_size=128, nb_epoch=10, validation_data=(X_test, Y_test))

# Evaluate model
score, acc = model.evaluate(X_test, Y_test, batch_size=128)
    
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)

Train on 15104 samples, validate on 6474 samples
Epoch 1/10
15104/15104 [==============================] - 33s - loss: 0.3541 - acc: 0.8490 - val_loss: 0.2728 - val_acc: 0.8613
Epoch 2/10
15104/15104 [==============================] - 33s - loss: 0.2751 - acc: 0.8611 - val_loss: 0.2575 - val_acc: 0.8719
Epoch 3/10
15104/15104 [==============================] - 32s - loss: 0.2553 - acc: 0.8814 - val_loss: 0.2300 - val_acc: 0.9040
Epoch 4/10
15104/15104 [==============================] - 32s - loss: 0.2397 - acc: 0.8962 - val_loss: 0.2286 - val_acc: 0.9053
Epoch 5/10
15104/15104 [==============================] - 32s - loss: 0.2340 - acc: 0.9004 - val_loss: 0.2210 - val_acc: 0.9080
Epoch 6/10
15104/15104 [==============================] - 32s - loss: 0.2291 - acc: 0.9053 - val_loss: 0.2169 - val_acc: 0.9124
Epoch 7/10
15104/15104 [==============================] - 32s - loss: 0.2248 - acc: 0.9078 - val_loss: 0.2120 - val_acc: 0.9150
Epoch 8/10
15104/15104 [==============================]